In [1]:
# Setup
from kafka import KafkaConsumer
import json
from pprint import pprint
from pymongo import MongoClient

In [2]:
def check_tabulation_and_newline(data):
    if type(data) == str:
        return data.replace("\t", " ").replace("\n", " ")
    else:
        return data

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

myclient = MongoClient('mongo', 27017, username='admin', password='DataMan2019!')
mydb = myclient["PFE_scrape"]
mycol = mydb["Track_test_25p"]

# We join (create) a consumer group for dynamic partition assignment and offset commits.
# If we run multile consumers with the same group_id only one of them will
# consume the message.
# Also, we pass a lambda function as value deserializer.
# This means the message we receive will be interpreted as a JSON value
# using the lambda function.
consumer = KafkaConsumer(
  bootstrap_servers=["kafka:9092"],
  auto_offset_reset="latest",
  value_deserializer=lambda v: json.loads(v.decode("utf-8")))

# We subscribe to a topic
consumer.subscribe(["pfe_scrape"])

# We "continuously stream" from the consumer object
#with open("pfe.tsv", "a") as file:
try:
    for msg in consumer:
        data = msg.value
        data = {key:check_tabulation_and_newline(data[key]) for key in data}
        #file.write(f"{data['Price of PFE stock']}\n")
        mycol.insert_one(data)
except:
    print("An exception occurred")
    #pass

# NOW YOU HAVE TO STOP THE CELL IF YOU ARE USING THE NOTEBOOK OR PRESS CTRL + C
# TO STOP THE CONSUMER